Read in the site data including measurement height, canopy height, latitude, longitude, and IGBP land cover class.

In [221]:
import pandas as pd

info = pd.read_csv('Fluxnet_measurement_canopy_heights.csv')
info.head()

,Site_name,IGBP,Lat,Lon,Year,M_height,C_height,a,balance,c,d,e,f,g,h,i,j
0,CN-Gebi.,BSV,42.001,101.137,2014,4.6,0.0,18.45,0.83,15.40,22.83,24.70,15.48,0.54,3.91,17.09,0.53
1,CN-Shens,BSV,38.789,100.493,2014,4.6,0.0,20.70,0.80,15.97,22.71,21.42,15.98,2.85,8.57,16.43,1.91
2,BE-Lon,CRO,50.552,4.746,2004~2014,2.7,0.5,20.91,0.80,14.23,14.53,15.58,14.23,2.72,0.34,7.45,2.89
3,CN-Daman,CRO,38.856,100.372,2014,4.5,2.0,13.61,0.93,17.60,17.49,14.82,17.76,6.08,5.75,0.13,9.27
4,DE-Geb,CRO,51.100,10.914,2001~2014,6.0,1.0,7.91,0.92,18.38,18.29,19.48,18.82,5.44,4.23,9.45,10.20


Load in the data from all the fluxnet sites in the project. Also, filter the data to remove some obviously erroneous values. Data are loaded into a dataframe, and then this is concatenated with the site-information dataframe from the first step. Now the DataFrame df contains all the usable data.

In [238]:
import os, zipfile
import pandas as pd, numpy as np
from matplotlib import pyplot as plt
import scipy
from scipy.optimize import fsolve
from math import sqrt
from numpy.lib.scimath import log

target='FULLSET_MM'

#dir_name = r'D:/My Drive/FLUXNET/'
stations = []
cover = []
data=pd.DataFrame()
total_sites=[]
for item in os.listdir(): # loop through items in dir
    if target in item:   #item is a single site's monthly FULLSET file from fluxnet
        site = item[4:10]
        total_sites.append(site)
        site_df_pre=pd.read_csv(item) ###Reads in the csv with all of "item"s data
        #Select only the columns we want to keep:  Use TIMESTAMP_START for weekly
        site_df_temp=site_df_pre.copy()
        if 'SWC_F_MDS_1' not in site_df_temp.columns: #use 'TIMESTAMP_START' for WW. Use "TIMESTAMP" otherwise
            site_df=site_df_temp[['TIMESTAMP', 'TA_F','VPD_F', 'PA_F', 'WS_F', 'NETRAD','G_F_MDS',\
                             'LE_F_MDS', 'H_F_MDS', 'USTAR']].copy()
            site_df['SWC_F_MDS_1']=-9999
        else:
            site_df=site_df_temp[['TIMESTAMP', 'TA_F','VPD_F', 'PA_F', 'WS_F', 'NETRAD','G_F_MDS',\
                             'LE_F_MDS', 'H_F_MDS', 'USTAR', 'SWC_F_MDS_1', 'LE_F_MDS_QC']].copy()
        site_df['G_F_MDS']=np.where(site_df_temp.G_F_MDS>-99, site_df_temp.G_F_MDS, 0)
        #site_df=site_df[['TIMESTAMP', 'TA_F','VPD_F', 'PA_F', 'WS_F', 'NETRAD', 'G_F_MDS',\
        #                     'LE_F_MDS', 'H_F_MDS']]
        site_df['G_F_MDS']=np.where(site_df.G_F_MDS>-99, site_df.G_F_MDS, 0)
        site_df['Site']=site
        land_cover=info[info.Site_name==site].IGBP
        site_df['IGBP']=land_cover.values[0]
        zt=info[info.Site_name==site].M_height
        site_df['zt']=zt.values[0]
        canop=info[info.Site_name==site].C_height
        site_df['h']=canop.values[0]
        site_df.replace(-9999, np.nan)
        data = pd.concat([data, site_df])
data=data[(data.NETRAD - data.G_F_MDS)>0]
data=data[data.H_F_MDS>0]
data=data[data.LE_F_MDS>0]
df=data
print(len(df.NETRAD))
df.head()

9980


,TIMESTAMP,TA_F,VPD_F,PA_F,WS_F,NETRAD,G_F_MDS,LE_F_MDS,H_F_MDS,USTAR,SWC_F_MDS_1,LE_F_MDS_QC,Site,IGBP,zt,h
8,200109,27.570,13.447,100.756,1.342,148.700521,0.0,33.0782,87.9074,0.335391,3.7,0.796528,AU-How,WSA,23.0,15.0
10,200111,28.716,7.516,100.504,1.073,170.314583,0.0,107.4380,41.9754,0.314733,8.0,0.998611,AU-How,WSA,23.0,15.0
11,200112,28.249,7.845,100.423,1.248,165.013441,0.0,125.9420,33.8638,0.379836,10.8,0.999328,AU-How,WSA,23.0,15.0
19,200208,24.175,15.869,100.852,1.709,127.253767,0.0,41.2987,70.0827,0.350694,5.8,0.740591,AU-How,WSA,23.0,15.0
20,200209,26.403,14.416,100.771,0.921,140.871537,0.0,42.0905,70.9755,-9999.000000,9.2,0.162500,AU-How,WSA,23.0,15.0


Make list of all sites that appear in df along with their other characteristics. Put it into a new dataframe, Site_Summary.

In [223]:
stations=pd.unique(df.Site)
Lat=[]
Lon=[]
measure_height=[]
canopy_height=[]
land_cover=[]
Column_names=['Site', 'Lat', 'Lon','Measure_height (m)', 'canopy_height (m)', 'IGBP_Class']
for station in stations:
    Lat.append(info[info.Site_name==station]['Lat'].values[0])
    Lon.append(info[info.Site_name==station]['Lon'].values[0])
    measure_height.append(info[info.Site_name==station]['M_height'].values[0])
    canopy_height.append(info[info.Site_name==station]['C_height'].values[0])
    land_cover.append(info[info.Site_name==station].IGBP.values[0])
Site_summary = pd.DataFrame(list(zip(stations, Lat, Lon, measure_height, canopy_height, land_cover)), columns=Column_names)



Site_summary.head()

,Site,Lat,Lon,Measure_height (m),canopy_height (m),IGBP_Class
0,AU-How,-12.494,131.152,23.0,15.0,WSA
1,AU-Rig,-36.650,145.576,5.0,0.4,GRA
2,AU-Stp,-17.151,133.350,4.2,0.2,GRA
3,AU-Tum,-35.657,148.152,70.0,40.0,EBF
4,AU-Wac,-37.426,145.188,95.0,70.0,EBF


Functions needed for the calculations

In [224]:
def estar(T):
  #Saturated vapor pressure (Pa) as a function of temp (C)
  # formulas from appendix of Andreas et al., 2013
  Aw, Bw, Ai, Bi = 17.502, 240.97, 22.452, 272.55 #
  estar = np.where(T>=0, 6.1121*1.004*np.exp(Aw*T/(Bw+T)), 6.1115*1.004*np.exp(Ai*T/(Bi+T)))
  return estar*100

def lv(T):
  #Latent heat of vaporization (J/kg) as a funcion of T (C) 
  #formulas from appendix of Andreas et al., 2013
  lv = np.where(T>=0, (25-0.02274*T)*100000, (28.34-0.00149*T)*100000)
  return lv

def Delta(T):
  #Slope of saturated vapor pressure curve at temp T
  #formulas from appendix of Andreas et al., 2013
  Aw, Bw, Ai, Bi = 17.502, 240.97, 22.452, 272.55
  D = np.where(T>=0, estar(T)*(Aw*Bw/(Bw+T)**2), estar(T)*(Ai*Bi/(Bi+T)**2))
  return D

def LEpenman(Rn, T, fu, ea, gamma):
  #Penman evap
  D = Delta(T)
  return (D/(D+gamma))*Rn+(gamma/(D+gamma))*lv(T)*fu*(estar(T)-ea)

def fu_z0(u, zu, zT, Ta, h, c1, c2, c3):
    #wind function using MOS theory
    z0=h/c1
    d0 = c2*h    #z0*0.67/0.1
    z0v = z0 / c3
    return 0.622 * 0.4**2 * u / (287.04 * (Ta + 273.15) * log((zT - d0) / z0v) * log((zu - d0) / z0))

def fu_pen(u, zu, h):
  #Wind function using the original penman formulation
  u2=u*(2/(zu))**(1/7)
  fu=0.26*(1+0.54*u2)/(100*24*60*60)
  return fu

def mean_abs_error(x,y):
    return np.nanmean(np.absolute(x-y))

def mean_squared_error(x,y):
    return np.mean((x-y)**2)

def rmserror(x,y):
  return np.sqrt(np.nanmean((x-y)**2))

def nse(predictions, targets):
    #Nash-Sutcliffe efficiency 
    return (1-(np.nansum((predictions-targets)**2)/np.nansum((targets-np.nanmean(targets))**2)))

Give variables more memorable names

In [225]:
df['u']=df.WS_F
df['Rn_G']=df.NETRAD-df.G_F_MDS
df['LEref']=df.Rn_G-df.H_F_MDS #df.Rn_G/(1+df.H_F_MDS/df.LE_F_MDS)     #df.Rn_G-df.Href
df['Href']=df.Rn_G-df.LEref
df['zu']=df.zt
df['zT']=df.zt
df['Ta']=df.TA_F+9.81*df.zT/1005
df['z0']=df.h*0.123   #df.h*0.123
df['d0']=df.h*0.67
df['z0v']=df.z0/10  #df.z0*0.1
df['p']=df.PA_F*1000
df['ea']=estar(df.Ta)-df.VPD_F*100
cp=1005
eps=0.622
df['gamma']=cp*df.p/(eps*lv(df.Ta))
df['Lv']=lv(df.Ta)
df['DeltaTa']=Delta(df.Ta)

In [226]:
df['fu'] =fu_z0(df.u, df.zu, df.zT, df.Ta, df.h, 8, .67, 10)
df['Ep']=LEpenman(df.Rn_G, df.Ta, df.fu, df.ea, df.gamma)

Solve numerically (using fsolve) for the wet surface ground temperature T0. This is done with a for loop.

In [227]:
T0pz0_trial = np.zeros((len(df.u)))
for n in range(len(df.u)):
    def Twspz0(Ts):
        return df['Rn_G'].values[n]/df['Ep'].values[n]-1\
            -df['gamma'].values[n]*(Ts-df['Ta'].values[n])*(estar(df['Ta'].values[n])-df['ea'].values[n])
    T0pz0_trial[n]=fsolve(Twspz0, df['Ta'].values[0])
df['T0']=T0pz0_trial
df.T0=np.where(df.T0 > df.Ta, df.T0, df.Ta)

C:\Users\rcrago\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone-2\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [228]:
df['Ee']=df.Rn_G * Delta(df.T0) / (Delta(df.T0) + df.gamma)
df['Tdry'] = df.Ta + df.ea / df.gamma
df['Ddry'] = Delta(df.Tdry)
df['Emax']=(df.Ddry/(df.Ddry+df.gamma))*df.Rn_G + (df.gamma/(df.Ddry+df.gamma))*lv(df.Ta)*df.fu*estar(df.Tdry)

Filter out all non-saturated surfaces using LEref>0.9*LEp

In [229]:
df_wet = df[df.LEref/df.Ep>.90]    # Epfu, Epz0,E0w
pd.unique(df_wet['IGBP'])
print('wet surface months: ', len(df_wet.Site))
print('Number of sites with wet surface evaporation: ', len(df_wet.Site.unique()))
print('wet surface cover classes', df_wet.IGBP.unique())
print(len(df_wet.Site.unique()))
wet_sites = []

wet surface months:  430
Number of sites with wet surface evaporation:  50
wet surface cover classes ['GRA' 'CRO' 'ENF' 'OSH' 'DBF' 'WET' 'EBF']
50


Make graphs showing different thresholds for wet surface evaporation. These graphs show that T=0.90 is a good compromise--there are still a large number of points, but LE is very close to LEp for these.

In [230]:
from scipy.stats import linregress
ranges=[[.95, 1.05], [0.9, 1.1], [0.85, 1.15], [0.95, 10], [0.9,10], [0.85, 10]]
ranges2=[[.95, 1.05], [0.9, 1.1], [0.85, 1.15], [0.95, '-'], [0.9, "-"], [0.85, '-']]
fig, axs = plt.subplots(3,2,sharex=True, sharey=True)
fig.set_figheight(12)
fig.set_figwidth(12)
pt_size=5
for row in range(3): # is the row
    for col in range(2):
        dftrial=df.copy()
        dftrial=dftrial[(dftrial.LEref/dftrial.Ep>ranges[row+3*col][0]) & (dftrial.LEref/dftrial.Ep<ranges[row+3*col][1])]
        x=dftrial.LEref
        y=dftrial.Ep
        axs[row,col].scatter(x,y, s= pt_size, c='black')
        axs[row,col].plot([0,250], [0,250], c='red', linestyle='dashed')
        s1, int1, r1, p1, se1 = linregress(x, y)
        axs[row,col].plot((5,250),(s1*5+int1, s1*250+int1), c= 'blue', linestyle='dotted')
        axs[row, col].set_xlabel('LE$_r$$_e$$_f$ W m$^-$$^2$')
        axs[row,col].set_ylabel('LE$_p$ W m$^-$$^2$')
        axs[row,col].set_xlim([0,275])
        axs[row,col].set_ylim([0,275])
        text0='panel '+ str(2*row+col+1)
        text1= 'range=' + str(ranges2[row+3*col]) + '; RMSD='+ str(round(rmserror(x,y),3)) 
        text2= "S="+ str(round(s1, 3))+ '; I='+str(round(int1,3)) + "; R=" + str(round(r1,3))+'; SE='+str(round(se1,3))
        text3= "NSE=" + str(round(nse(x,y),3))+'; Number of points=' + str(round(len(x)))
        s=9.5
        axs[row,col].text(5,260, text0, fontsize=s)
        axs[row,col].text(5,240, text1, fontsize=s)
        axs[row,col].text(5,220,text2, fontsize=s)
        axs[row,col].text(5,200,text3, fontsize=s)
plt.show()    
    

In order to map the sites that have at least one measurement that qualifies as wet surface, and also to map all the other sites (with no wet surface measurements) we need lists of these sites along with their latitude and longitude.

In [231]:
nonwet_sites_lat=[]
nonwet_sites_lon=[]
wet_sites_lat=[]
wet_sites_lon=[]
for n, station in enumerate(stations):
    Site_latitude = Site_summary[Site_summary['Site']==station]['Lat'].values.tolist()[0]
    Site_longitude = Site_summary[Site_summary['Site']==station]['Lon'].values.tolist()[0]
    if station in df_wet.Site.unique():
        wet_sites_lat.append(Site_latitude)
        wet_sites_lon.append(Site_longitude)
    else:
        nonwet_sites_lat.append(Site_latitude)
        nonwet_sites_lon.append(Site_longitude)
print(len(nonwet_sites_lon))
print(len(wet_sites_lon))

121
50


Plot sites with some wet surface values and then plot sites with no wet surface values. Note that geopandas is not part of the normal python istallation, so it is installed using pip.

In [239]:
!pip install geopandas

import geopandas as gpd
from shapely.geometry import Point
from geopandas import GeoDataFrame

geometry_wet = [Point(xy) for xy in zip(wet_sites_lon, wet_sites_lat)]
geometry_dry = [Point(xy) for xy in zip(nonwet_sites_lon, nonwet_sites_lat)]
gdfwet=GeoDataFrame(geometry=geometry_wet)
gdfdry=GeoDataFrame(geometry=geometry_dry)
world=gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdfwet.plot(ax=world.plot(figsize=(10,10), color='tan'), marker='o', color='blue', markersize=15)
gdfdry.plot(ax=world.plot(figsize=(10,10), color='tan'), marker='o', color='red', markersize=15)
plt.show()

Initialize a process to find tuned parameters for alpha.

In [233]:
from scipy.stats import linregress
import random

E=df_wet.LEref
R=df_wet.Rn_G
T=df_wet.T0  #df_wet.T0w  #df_wet.T0pz0  #df_wet.T0pfu
g=df_wet.gamma
fu=df_wet.fu #fuz0  #fupen
e_a=df_wet.ea
T_a=df_wet.Ta

D=Delta(T)
Ee=D*R/(D+g)
Emax=lv(T)*fu*estar(T)

alpha=E*(D+g)/(D*R)

rms1best=1000
rms2best=1000
rms3best=1000
rms4best=1000
rms5best=1000
rms6best=1000
rms7best=1000
rms8best=2000

Process to find tuned parameter values--those values that minimze RMSD of alpha or of LEPT. This is found by randomly selecting values in a realistic range for each parameter, finding the RMSD, and selecting the parameter value that gave the lowest RMSD value. 3000 random selections are made.

In [234]:
for n in range(3000):
    if n in [10, 500, 1000, 1500, 2000, 2500]: print(n)
    RH=random.uniform(0, 1)
    aA=random.uniform(0, 1)
    alp=random.uniform(1, 1.6)
    m_alph=random.uniform(0,1)
    alpha_RH = 1 + (g/Delta(T))*lv(T)*fu*estar(T)*(1-RH)/R
    alpha_RH = np.where(alpha_RH>(1+g/D), 1+g/D, alpha_RH)
    alpha_A=(D+g)/(D+aA*g)
    alpha_m=1+g*m_alph/D
    LEPT_const=alp*D*R/(D+g)
    LEPT_RH=alpha_RH*D*R/(D+g)
    LEPT_A=alpha_A*D*R/(D+g)
    LEPT_const=alp*D*R/(D+g)
    LEPT_m=alpha_m*D*R/(D+g)
    alph=np.zeros((len(df_wet.Ta)))+alp
    rms1=rmserror(alpha, alpha_RH)
    rms2=rmserror(alpha, alpha_A)
    rms3=rmserror(alpha,alph)
    rms4=rmserror(alpha,alpha_m)
    rms5=rmserror(E, LEPT_RH)
    rms6=rmserror(E, LEPT_A)
    rms7=rmserror(E, LEPT_const)
    rms8=rmserror(E, LEPT_m)
    
    
    if rms1 < rms1best:#alpha_RH 
        rms1best=rms1
        RH_best=RH
    if rms2 <rms2best: #alpha_A
        rms2best=rms2
        aA_best=aA
    if rms3<rms3best: #alpha_c
        rms3best=rms3
        alp_best=alp
    if rms4<rms4best: #alpha_m
        rms4best=rms4
        m_best=m_alph
    if rms5<rms5best: #LEPT_RH
        rms5best=rms5
        RH_Ebest=RH  
    if rms6<rms6best: #LEPT_Andreas
        rms6best=rms6 
        aA_Ebest=aA
    if rms7<rms7best: #LEPT_alpha_const
        rms7best=rms7
        alp_Ebest=alp
    if rms8<rms8best: #LEPT_m
        rms8best=rms8
        m_Ebest=m_alph



10
500
1000
1500
2000
2500


Plots of model performace with the tuned parameters, along with performance statistics.

In [235]:
alpha_RH = 1 + (g/D)*lv(T)*fu*estar(T)*(1-RH_best)/R
alpha_RH = np.where(alpha_RH>(1+g/D), 1+g/D, alpha_RH)
alpha_A=(D+g)/(D+aA_best*g)
alpha_m=1+g*m_best/D
alpha_RH_E = 1 + (g/D)*lv(T)*fu*estar(T)*(1-RH_Ebest)/R
alpha_RH_E=np.where(alpha_RH_E > 1+g/D, 1+g/D, alpha_RH_E)
LEPT_RH= alpha_RH_E*D*R/(D+g)
LEPT_A = ((D+g)/(D+aA_Ebest*g))*D*R/(D+g)
LEPT_alphabest =alp_Ebest*D*R/(D+g)
LEPT_mbest = (1+g*m_Ebest/D)*D*R/(D+g)

x1=alpha
y1=alpha_RH
x2=alpha
y2=alpha_A
x3=alpha
y3=alp_best*alpha/alpha
x4=alpha
y4=alpha_m
s1, int1, r1, p1, se1 = linregress(x1, y1) 
s2, int2, r2, p2, se2 = linregress(x2, y2)       
s3, int3, r3, p3, se3 =linregress(x3, y3)
s4, int4, r4, p4, se4 =linregress(x4, y4)
NSE1, NSE2, NSE3, NSE4 = nse(x1, y1), nse(x2, y2), nse(x3, y3), nse(x4, y4)
#print(np.max(y2-y3))

x5=E
y5=LEPT_RH
x6=E
y6=LEPT_A
x7=E
y7=LEPT_alphabest
x8=E
y8=LEPT_mbest
s5, int5, r5, p5, se5 = linregress(x5, y5)
s6, int6, r6, p6, se6 = linregress(x6, y6)
s7, int7, r7, p7, se7 = linregress(x7, y7)
s8, int8, r8, p8, se8 = linregress(x8, y8)
NSE5, NSE6, NSE7, NSE8 = nse(x5, y5), nse(x6, y6), nse(x7, y7), nse(x8, y8)

text1 = "RH-method: " + 'RH='+ str(round(RH_best, 2)) + '; RMSD=' + str(round(rmserror(x1, y1), 2))\
        + '; R=' + str(round(r1, 2)) + '; S='+str(round(s1, 2))+'; I=' + str(round(int1, 2))+"; NSE="+str(round(NSE1,2))
text2 = 'a$_A$-method: '+'a$_A$=' + str(round(aA_best, 2))+'; RMSD='+str(round(rmserror(x2,y2),2))\
        +'; R='+ str(round(r2,2))+'; S='+str(round(s2,2))+'; I='+str(round(int2, 2))+"; NSE="+str(round(NSE2,2))
text3 = r'$\alpha$$_c$-method: '+'alpha=' + str(round(alp_best, 2))+'; RMSD='+str(round(rmserror(x3,y3),2))\
        +'; R=0 ;'+ 'S='+'0;'+ 'I='+str(round(int3, 2))+"; NSE=0"
text4 = 'm-method: ''m =' + str(round(m_best, 2))+'; RMSD='+str(round(rmserror(x4,y4),2))\
        +'; R='+ str(round(r4,2))+'; S='+str(round(s4,2))+'; I='+str(round(int4, 2))+"; NSE="+str(round(NSE4,2))
text5 = "RH-method: " + 'RH = '+ str(round(RH_Ebest, 2)) + '; RMSD= ' + str(round(rmserror(x5, y5), 2)) + ' W/m$^2$'\
        + '; R = ' + str(round(r5, 2)) + '; S= ' + str(round(s5, 2)) + "; I= " + str(round(int5, 2)) + ' W/m$^2$'+"; NSE="+str(round(NSE5,2))
text6 = 'a$_A$-method: '+'a$_A$ = ' + str(round(aA_Ebest, 2))+'; RMSD= '+str(round(rmserror(x6,y6),2))+ ' W/m$^2$'\
        +'; R= '+ str(round(r6,2))+'; S= '+str(round(s6,2))+'; I= '+str(round(int6, 2))+ ' W/m$^2$'+"; NSE="+str(round(NSE6,2))  
text7 = r'$\alpha$$_c$-method: '+'alpha = ' + str(round(alp_Ebest, 2))+'; RMSD= '+str(round(rmserror(x7,y7),2))+ ' W/m$^2$'\
        +'; R= '+ str(round(r7,2))+'; S= '+str(round(s7,2))+'; I= '+str(round(int7, 2))+ ' W/m$^2$'+"; NSE="+str(round(NSE7,2))
text8 = 'm-method: '+'m = ' + str(round(m_Ebest, 2))+'; RMSD= '+str(round(rmserror(x8,y8),2))+ ' W/m$^2$'\
        +'; R= '+ str(round(r8,2))+'; S= '+str(round(s8,2))+'; I= '+str(round(int8, 2))+ ' W/m$^2$'+"; NSE="+str(round(NSE8,2))
text9="Num=" + str(len(df_wet.Ta)) #+ '; IGBP=' + str([x for x in pd.unique(df_wet.IGBP)])

fs=12
msize=20

fig, (ax1, ax2) = plt.subplots(2,1)
fig.set_figheight(13)
fig.set_figwidth(10)
plt.rcParams.update({'font.size': fs})

ax1.scatter(x1, y1, marker='.', s=msize, c='black', label=r'$\alpha$$_R$$_H$')
ax1.scatter(x2, y2, marker='x', s=msize, c='red', alpha=0.4, label=r'$\alpha$$_a$$_A$')
ax1.scatter(x3,y3, marker=',', s=msize, c='brown', alpha=0.4, label=r'$\alpha$$_c$')
ax1.plot([1, 2.9], [1,2.9], linestyle='solid', c='gray', )
ax1.scatter(x4,y4, marker='o', s=msize, c='green', alpha=.2, label=r'$\alpha$$_m$')
ax1.set_xlabel(r'$\alpha$$_r$$_e$$_f$', fontsize=fs+4)
ax1.set_ylabel(r'$\alpha$$_e$$_s$$_t$', fontsize=fs+4)
ax1.set_xlim(1,2.5)
ax1.set_ylim(1,3.1)
ax1.text(1.05, 3.0, text3, fontsize=fs)
ax1.text(1.05, 2.9, text2, fontsize=fs)
ax1.text(1.05, 2.8, text1, fontsize=fs)
ax1.text(1.05, 2.7, text4, fontsize=fs)
ax1.text(1.05, 2.6, text9, fontsize=fs)
ax1.legend(loc='lower right')

ax2.scatter(x5, y5, marker='.', s=msize, c='black', label=r'$\alpha$$_R$$_H$')
ax2.scatter(x6, y6, marker='x', s=msize, c='red', alpha=0.4, label=r'$\alpha$$_a$$_A$')
ax2.scatter(x7, y7, marker=',', s=msize, c='blue', alpha=0.3, label=r'$\alpha$$_c$')
ax2.scatter(x8, y8, marker='o', s=msize, c='green', alpha=0.2, label=r'$\alpha$$_m$')
ax2.plot([1,250], [1,250], c='brown')
ax2.set_xlabel('LE$_r$$_e$$_f$ (W/m$^2$)', fontsize=fs+4)
ax2.set_ylabel('LE$_P$$_T$ (W/m$^2$)', fontsize=fs+4)
ax2.set_xlim(0,250)
ax2.set_ylim(0,320)
ax2.text(2, 308, text7, fontsize=fs)
ax2.text(2,291, text6, fontsize=fs)
ax2.text(2,274, text5, fontsize=fs)
ax2.text(2,257, text8, fontsize=fs)
ax2.text(2,239, text9, fontsize=fs)
ax2.legend(loc='lower right')
plt.show()

Similar random trials to get tuned parameter values for estimating LE with the CR. This process is mucn slower because all the data needs to be processed. Good results are obtainable with only 1000 iterations.

In [236]:
import random
import pandas as pd
from scipy import stats
from scipy.stats import linregress

best_rms=np.zeros((4))+10000
best_par=np.zeros((4))
LEmodel=np.zeros((4, len(df.Ta)))

def yvalues(Sd, Sw, xm, x):
  d=(Sd*xm - Sd + Sw*xm - Sw + 2)/(xm**3 - 3*xm**2 + 3*xm - 1)
  c=(-Sw*xm + Sw - d*xm**3 + 3*d*xm - 2*d - 1)/(xm**2 - 2*xm + 1)
  b=(-c*xm**2 + c - d*xm**3 + d - 1)/(xm - 1)
  a=1-(b+c+d)
  return a+b*x+c*x**2+d*x**3

iterations=3000
for a in range(iterations):
    if a in [1, 500, 1000, 2000, 2500]:
        print(a)
    alp=random.uniform(1, 1.6)
    aA=random.uniform(0, 1)
    RH=random.uniform(0,1)
    SD=1#random.uniform(0, 2)
    SW=1#random.uniform(0, 2)
    M=random.uniform(0, 1)
    for a in range(4):
        if a==0: #constant alpha
            par=alp
            alpha=alp
            x=alpha*df.Ee/df.Ep
            xmin=alp*df.Ee/df.Emax
        if a==1: #constant aA
            par=aA
            alpha=(Delta(df.T0)+df.gamma)/(Delta(df.T0)+aA*df.gamma)
            x=alpha*df.Ee/df.Ep
            xmin=alpha*df.Ee/df.Emax
        if a==2:  #constant RH
            par=RH
            alpha=1+(df.gamma/Delta(df.T0))*lv(df.Ta)*df.fu*estar(df.T0)*(1-RH)/df.Rn_G
            alpha=np.where(alpha>1+df.gamma/Delta(df.T0), 1+df.gamma/Delta(df.T0), alpha)
            x=alpha*df.Ee/df.Ep
            xmin=alpha*df.Ee/df.Emax
        if a==3: #constant M
            par=M
            alpha=(1+df.gamma*M/Delta(df.T0))
            x=alpha*df.Ee/df.Ep
            xmin=alpha*df.Ee/df.Emax           
        x=np.where(x<0,0.0001,x)
        x=np.where(x>1, 1, x)
        xmin=np.where(xmin<0, 0.0001, xmin)
        xmin=np.where(xmin>x, x, xmin)
        LEest=yvalues(SD, SW, xmin, x)*df.Ep
        rms=rmserror(df.LEref, LEest)
        if rms<best_rms[a]:
            best_rms[a]=rms
            best_par[a]=par
            LEmodel[a,:]=LEest
print(best_rms)
print(best_par)    

1


__main__:11: RuntimeWarning: divide by zero encountered in true_divide
__main__:12: RuntimeWarning: invalid value encountered in add


500
1000
2000
2500
[19.37382413 18.60957412 21.12573829 18.6451736 ]
[1.21852597 0.4304562  0.9643579  0.45099669]


Optimal parameter values used in CR estimates of LE

In [237]:
fig, axs = plt.subplots(2, 2)
fig.set_figheight(15)
fig.set_figwidth(15)
plt.rcParams.update({'font.size': 14})

version_b=[r'$\alpha$$_c$','a$_A$', 'RH', 'm'] 
sz=.3
for a in range(4):
    c1=[0,0,1,1]
    c2=[0,1,0,1]
    xy=pd.DataFrame(list(zip(df.LEref, LEmodel[a,:],)), columns=['x','y'])
    xy=xy.dropna()
    axs[c1[a],c2[a]].scatter(xy.x, xy.y, s=sz, c='black')
    axs[c1[a],c2[a]].plot([0, 200], [0, 200], c='red', marker=None, linestyle='dashed')
    axs[c1[a],c2[a]].set_xlim([0, 210])
    axs[c1[a],c2[a]].set_ylim([0,210])
    axs[c1[a],c2[a]].set_xlabel('LE$_r$$_e$$_f$ (W/m$^2$)', fontsize=14)
    axs[c1[a],c2[a]].set_ylabel('LE$_e$$_s$$_t$ (W/m$^2$)', fontsize=14)
    rms=rmserror(xy.x, xy.y)
    NSE=nse(xy.x, xy.y)
    S, I, R, p1, se1 = linregress(xy.x, xy.y)
    axs[c1[a],c2[a]].plot((0,200), (S*0+I,  S*200+I), marker=None, c='blue', linestyle='dotted')
    text1=version_b[a]+'-method: ' +  version_b[a] +'=' + str(round(best_par[a],2))
    text2='RMS' + '='+ str(round(rms,2))+ '(W/m$^2$)'+'; R='+str(round(R,2))
    text3 = 'NSE=' + str(round(NSE,2))
    text4='Slope=' + str(round(S,2)) + ' Int (W/m$^2$)=' + str(round(I,2))
    text5='y=X'+"; Num="+str(len(df.Ta))
    panel=['a', 'b', 'c', 'd']
    text6 = 'panel '+ str(panel[2*c1[a]+c2[a]])
    #text5='SD='+str(SD)+ "; SW=" + str(SW)
    axs[c1[a],c2[a]].text(1,190, text1)
    axs[c1[a],c2[a]].text(1,180, text2)
    axs[c1[a],c2[a]].text(1,170, text3)
    axs[c1[a],c2[a]].text(1,160, text4)
    axs[c1[a],c2[a]].text(1,150, text5)
    axs[c1[a],c2[a]].text(1,200, text6)
plt.show()